In [2]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="NvHkaNk0UepsPtmn2wZZ")
project = rf.workspace("syed-burhan-ahmed-ratws").project("cv-project-part-2")
version = project.version(4)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to CV-project-Part-2-4 in yolov11:: 100%|██████████| 594/594 [00:00<00:00, 11649.75it/s]


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="NvHkaNk0UepsPtmn2wZZ")
project = rf.workspace("marydhel-compaero").project("female-or-male")
version = project.version(3)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Female-or-Male-3 in yolov11:: 100%|██████████| 10650/10650 [00:01<00:00, 6962.30it/s]


In [6]:
import os
import shutil
import yaml

def merge_datasets_and_yaml(sources, dest):
    splits = ['train', 'valid', 'test']
    subdirs = ['images', 'labels']

    os.makedirs(dest, exist_ok=True)
    all_class_names = set()

    for split in splits:
        for sub in subdirs:
            dest_dir = os.path.join(dest, split, sub)
            os.makedirs(dest_dir, exist_ok=True)
            for src in sources:
                src_dir = os.path.join(src, split, sub)
                if os.path.exists(src_dir):
                    for fname in os.listdir(src_dir):
                        src_file = os.path.join(src_dir, fname)
                        dest_file = os.path.join(dest_dir, fname)
                        if os.path.exists(dest_file):
                            base, ext = os.path.splitext(fname)
                            new_name = f"{base}_{os.path.basename(src)}{ext}"
                            dest_file = os.path.join(dest_dir, new_name)
                        shutil.copy2(src_file, dest_file)

    # Merge class names from all YAMLs
    for src in sources:
        yaml_path = os.path.join(src, "data.yaml")
        if os.path.exists(yaml_path):
            with open(yaml_path, 'r') as f:
                data = yaml.safe_load(f)
                all_class_names.update(data.get("names", []))

    all_class_names = sorted(list(all_class_names))
    merged_yaml = {
        'train': os.path.join(dest, 'train', 'images'),
        'val': os.path.join(dest, 'valid', 'images'),
        'test': os.path.join(dest, 'test', 'images'),
        'nc': len(all_class_names),
        'names': all_class_names
    }

    with open(os.path.join(dest, 'data.yaml'), 'w') as f:
        yaml.dump(merged_yaml, f)

    print(f"✅ Merged dataset created at: {dest}")
    print(f"✅ Merged data.yaml includes {len(all_class_names)} classes: {all_class_names}")

# Example usage:
dataset1_path = '/content/CV-project-Part-2-4'
#dataset2_path = '/kaggle/working/Female-or-Male-3'
dataset2_path = '/content/Female-or-Male-3'
merged_path   = '/content/merged_dataset'


# merge_datasets_and_yaml([dataset1_path, dataset2_path, dataset3_path], merged_path)

merge_datasets_and_yaml([dataset1_path, dataset2_path], merged_path)



✅ Merged dataset created at: /content/merged_dataset
✅ Merged data.yaml includes 2 classes: ['Female', 'Male']


In [7]:
def write_clean_yaml(output_dir):
    yaml_content = """train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Female', 'Male']
"""
    output_path = os.path.join(output_dir, "data.yaml")

    with open(output_path, 'w') as f:
        f.write(yaml_content)

    print(f"✅ data.yaml saved at: {output_path}")

# Example usage
output_folder = "/content/merged_dataset"  # Change this as needed
write_clean_yaml(output_folder)


✅ data.yaml saved at: /content/merged_dataset/data.yaml


In [5]:
import shutil
shutil.rmtree('/content/CV-project-Part-2-5')

In [8]:
from ultralytics import YOLO

In [21]:

# Path to your data.yaml file
data_yaml_path = '/content/merged_dataset/data.yaml'

yolo_model = YOLO('/content/yolo11n.pt')
# model.model.head.reset_class(nc=2)

# # Access the detection head (last module in the model's module list)
# head = yolo_model.model.model[-1]

# # Change number of output classes to 2 (male, female)
# head.nc = 2


In [22]:
# # Freeze all layers except the detection head
# for param in yolo_model.model.model[:-1].parameters():
#     param.requires_grad = False
# for param in yolo_model.model.model[-1].parameters():
#     param.requires_grad = True

# Train for 30 epochs
results = yolo_model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=250,
    batch=16,
    # weight_decay=0.0005,
    patience=20,
    # optimizer='AdamW',
    # lr0=0.001,
    name='face-gender-detection',
    project='/content/yolo_runs',
    verbose=True,
    device='cuda',
    save=True
)

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=250, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=face-gender-detection3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=

train: Scanning /content/merged_dataset/train/labels.cache... 4917 images, 123 backgrounds, 0 corrupt: 100%|██████████| 4917/4917 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 190.7±90.5 MB/s, size: 8.5 KB)


val: Scanning /content/merged_dataset/valid/labels.cache... 444 images, 7 backgrounds, 0 corrupt: 100%|██████████| 444/444 [00:00<?, ?it/s]


...

      50/50     0.926G     0.6091     0.2972     0.9949          5        256: 100%|██████████| 308/308 [00:46<00:00,  6.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  6.13it/s]

                   all        444        440      0.943      0.941      0.958      0.714



50 epochs completed in 0.705 hours.
Optimizer stripped from /content/yolo_runs/face-gender-detection3/weights/last.pt, 5.4MB
Optimizer stripped from /content/yolo_runs/face-gender-detection3/weights/best.pt, 5.4MB

Validating /content/yolo_runs/face-gender-detection3/weights/best.pt...
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:03<00:00,  3.54it/s]


                   all        444        440       0.94      0.939      0.966      0.726
                Female        239        241      0.943      0.969      0.978      0.747
                  Male        198        199      0.936       0.91      0.955      0.704
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /content/yolo_runs/face-gender-detection3


In [25]:
import shutil

# Path to the folder you want to zip
folder_to_zip = "/content/yolo_runs/face-gender-detection3"
zip_output_path = "/content/yolo_runs/face-gender-detection3.zip"

# Create ZIP file
shutil.make_archive(base_name=zip_output_path.replace(".zip", ""), format='zip', root_dir=folder_to_zip)

print(f"✅ Zipped folder saved as: {zip_output_path}")


✅ Zipped folder saved as: /content/yolo_runs/face-gender-detection3.zip


In [15]:
def detect_and_classify_faces(image_path, output_path='annotated_output.jpg'):
    # Read image
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        raise FileNotFoundError(f"❌ Could not read image: {image_path}")

    img_height, img_width = img_cv.shape[:2]
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

    # Run YOLOv8 face detection
    results = yolo_model.predict(
    img_rgb,
    conf=0.1,
    iou=0.5,
    imgsz=1024,
    verbose=False,
    agnostic_nms=True
)

    boxes = results[0].boxes

    if boxes is None or len(boxes) == 0:
        print("❌ No faces detected.")
        return

    print(f"\n✅ Detected {len(boxes)} face(s):\n")

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy().astype(int)
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(img_width, x2), min(img_height, y2)

        gender = int(boxes.cls[i])
        confidence = float(boxes.conf[i])
        color = GENDER_COLORS.get(gender, (255, 255, 255))

        face = img_rgb[y1:y2, x1:x2]
        if face.size == 0:
            continue

        # --- Face Classification ---
        transformed = transform(image=face)
        face_tensor = transformed['image'].unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            output = classifier_model(face_tensor)
            pred_class = int(torch.argmax(output, dim=1).item())

        label = FACE_CLASSES.get(pred_class, 'Unknown')
        gender_text = gender_map.get(gender, 'Unknown')
        text = f"{gender_text}, {label}, {confidence:.2f}"

        print(f"Face {i+1}: {text}")

        # --- Draw box + label ---
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        margin = 4
        (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, thickness)
        text_x = max(min(x1, img_width - text_w - margin), margin)
        text_y = y1 - margin if y1 - text_h - margin > margin else y2 + text_h + margin
        text_y = min(text_y, img_height - margin)

        # Draw background + text
        cv2.rectangle(img_cv,
                      (text_x - margin, text_y - text_h - margin),
                      (text_x + text_w + margin, text_y + margin),
                      color, -1)
        cv2.putText(img_cv, text, (text_x, text_y),
                    font, font_scale, (255, 255, 255), thickness)

        # Draw bounding box
        cv2.rectangle(img_cv, (x1, y1), (x2, y2), color, 1)

    # Save and display
    cv2.imwrite(output_path, img_cv)
    print(f"\n📸 Saved annotated image to: {output_path}")
    cv2_imshow(img_cv)

In [23]:
#!pip install ultralytics
import os
import cv2
import zipfile
import shutil
import torch
import timm
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from torchvision import transforms, datasets
from torchvision.models import efficientnet_b0
from torch.utils.data import DataLoader, random_split, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR
import os
import cv2
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import torch.nn as nn

# --- CONFIGURATION ---
TEST_DIR = '/content/test_img'
YOLO_MODEL_PATH = '/content/best.pt'
CLASSIFIER_MODEL_PATH = '/content/CV_Project_Part1.pth'
OUTPUT_FOLDER = '/content/annotated prediction'
IMAGE_SIZE = (224, 224)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- 1. Load YOLO Model ---
# yolo_model = YOLO(YOLO_MODEL_PATH)

# --- 2. Define and Load Classifier Model ---
def create_model():
    model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    in_f = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(in_f, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 16)  # 15 celebrities + 1 for others
    )
    return model

classifier_model = create_model().to(DEVICE)
classifier_model.load_state_dict(torch.load(CLASSIFIER_MODEL_PATH, map_location=DEVICE))
classifier_model.eval()

# --- 3. Preprocessing for classifier ---
transform = A.Compose([
    A.Resize(*IMAGE_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# === Class Labels ===
FACE_CLASSES = {
    0:  "Iqra Aziz",
    1:  "Humayun Saeed",
    2:  "Kubra Khan",
    3:  "Mahira Khan",
    4:  "Naseem Shah",
    5:  "Noman Ijaz",
    6:  "Neelam Muneer",
    7:  "Ramsha Khan",
    8:  "Sajal Aly",
    9:  "Shaheen Afridi",
    10: "Atif Aslam",
    11: "Fahad Mustafa",
    12: "Fawad Khan",
    13: "Hamza Ali Abbasi",
    14: "Hania Amir",
    -1: "Others"
}
gender_map = {0: "Female", 1: "Male"}
GENDER_COLORS = {0: (0, 255, 0), 1: (255, 0, 0)}  # Green: Female, Blue: Male



In [24]:
detect_and_classify_faces("/content/WhatsApp Image 2025-07-09 at 15.55.07_a774f1fb.jpg")


✅ Detected 7 face(s):

Face 1: Male, Unknown, 0.69
Face 2: Female, Kubra Khan, 0.65
Face 3: Female, Unknown, 0.62
Face 4: Female, Unknown, 0.61
Face 5: Male, Unknown, 0.51
Face 6: Female, Hania Amir, 0.40
Face 7: Male, Noman Ijaz, 0.10

📸 Saved annotated image to: annotated_output.jpg


In [30]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    """
    Unzips a zip file to the specified directory.

    Args:
        zip_path (str): Path to the .zip file.
        extract_to (str): Directory where files will be extracted.
    """
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"Zip file not found: {zip_path}")
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted all files to {extract_to}")


In [31]:
unzip_file("/content/Test Images.zip","/content/test_images")

Extracted all files to /content/test_images


In [37]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import torch.nn as nn

# --- CONFIGURATION ---
TEST_DIR = '/content/check'
YOLO_MODEL_PATH = '/kaggle/working/yolo_runs/face-gender-detection/weights/best.pt'
CLASSIFIER_MODEL_PATH = '/content/CV_Project_Part1.pth'
OUTPUT_FOLDER = '/content/annotated_predictions'
IMAGE_SIZE = (224, 224)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- 1. Load YOLOv8 Model ---
# yolo_model = YOLO(YOLO_MODEL_PATH)

# --- 2. Define and Load Classifier Model ---
def create_model():
    model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    in_f = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(in_f, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 16)  # 15 celebrities + 1 for others
    )
    return model

classifier_model = create_model().to(DEVICE)
classifier_model.load_state_dict(torch.load(CLASSIFIER_MODEL_PATH, map_location=DEVICE))
classifier_model.eval()

# --- 3. Preprocessing for classifier ---
transform = A.Compose([
    A.Resize(*IMAGE_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# --- 4. Class and Gender Mapping ---
class_names = {
    0:  "Iqra Aziz", 1:  "Humayun Saeed", 2:  "Kubra Khan", 3:  "Mahira Khan",
    4:  "Naseem Shah", 5:  "Noman Ijaz", 6:  "Neelam Muneer", 7:  "Ramsha Khan",
    8:  "Sajal Aly", 9:  "Shaheen Afridi", 10: "Atif Aslam", 11: "Fahad Mustafa",
    12: "Fawad Khan", 13: "Hamza Ali Abbasi", 14: "Hania Amir", -1: "Other"
}
gender_map = {0: "Female", 1: "Male"}
GENDER_COLORS = {0: (0, 255, 0), 1: (255, 0, 0)}

# --- 5. Batch Inference and Annotation ---
for fn in tqdm(os.listdir(TEST_DIR)):
    if not fn.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(TEST_DIR, fn)
    img_cv = cv2.imread(img_path)
    if img_cv is None:
        print(f"Warning: Could not read image {fn}")
        continue
    img_height, img_width = img_cv.shape[:2]
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

    # YOLOv8 Inference with class-agnostic NMS
    results = yolo_model.predict(
        img_rgb, conf=0.1, iou=0.5, imgsz=1024, verbose=False, agnostic_nms=True
    )
    boxes = results[0].boxes

    if boxes is None or len(boxes) == 0:
        print(f"No faces detected in {fn}.")
        cv2.imwrite(os.path.join(OUTPUT_FOLDER, fn), img_cv)
        continue

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy().astype(int)
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(img_width, x2), min(img_height, y2)
        gender = int(boxes.cls[i])
        color = GENDER_COLORS.get(gender, (255, 255, 255))

        face = img_rgb[y1:y2, x1:x2]
        if face.size == 0:
            continue

        # Preprocess and classify face
        transformed = transform(image=face)
        face_tensor = transformed['image'].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = classifier_model(face_tensor)
            pred_class = int(torch.argmax(output, dim=1).item())
        label = class_names.get(pred_class, 'Other')
        gender_text = gender_map.get(gender, 'Unknown')
        text = f"{gender_text}, {label}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        thickness = 2

        # Use the top-left corner for text placement
        x_text, y_text = int(x1), int(y1)
        (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
        y_text = max(y_text - 10, text_height + 5)

        # Draw filled rectangle for text background
        cv2.rectangle(
            img_cv,
            (x_text, y_text - text_height - baseline),
            (x_text + text_width, y_text + baseline),
            (0, 255, 0) if gender == 0 else (255, 0, 0),
            thickness=cv2.FILLED
        )
        # Draw text
        cv2.putText(
            img_cv,
            text,
            (x_text, y_text),
            font,
            font_scale,
            (255, 255, 255),
            thickness
        )
        # Draw bounding box
        cv2.rectangle(
            img_cv,
            (x1, y1),
            (x2, y2),
            (0, 255, 0) if gender == 0 else (255, 0, 0),
            2
        )

    # Save annotated image
    out_path = os.path.join(OUTPUT_FOLDER, fn)
    cv2.imwrite(out_path, img_cv)

print(f"✅ Annotated images saved in: {OUTPUT_FOLDER}")


100%|██████████| 6/6 [00:00<00:00,  7.28it/s]

✅ Annotated images saved in: /content/annotated_predictions


In [39]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import torch.nn as nn

# --- CONFIGURATION ---
TEST_DIR = '/content/test_images'
YOLO_MODEL_PATH = '/content/yolo_runs/face-gender-detection3/weights/best.pt'
CLASSIFIER_MODEL_PATH = '/content/CV_Project_Part1.pth'
OUTPUT_FOLDER = '/content/annotated_predictions_finall'
IMAGE_SIZE = (224, 224)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- 1. Load YOLO Model ---
yolo_model = YOLO(YOLO_MODEL_PATH)

# --- 2. Define and Load Classifier Model ---
def create_model():
    model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    in_f = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(in_f, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 16)  # 15 celebrities + 1 for others
    )
    return model

classifier_model = create_model().to(DEVICE)
classifier_model.load_state_dict(torch.load(CLASSIFIER_MODEL_PATH, map_location=DEVICE))
classifier_model.eval()

# --- 3. Preprocessing for classifier ---
transform = A.Compose([
    A.Resize(*IMAGE_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# --- 4. Class and Gender Mapping ---
class_names = {
    0:  "Iqra Aziz",
    1:  "Humayun Saeed",
    2:  "Kubra Khan",
    3:  "Mahira Khan",
    4:  "Naseem Shah",
    5:  "Noman Ijaz",
    6:  "Neelam Muneer",
    7:  "Ramsha Khan",
    8:  "Sajal Aly",
    9:  "Shaheen Afridi",
    10: "Atif Aslam",
    11: "Fahad Mustafa",
    12: "Fawad Khan",
    13: "Hamza Ali Abbasi",
    14: "Hania Amir",
    -1: "Others"
}
class_names[-1] = "Other"
gender_map = {0: "Female", 1: "Male"}

# --- 5. Inference Loop with Axis-Aligned Bounding Boxes ---
for fn in tqdm(os.listdir(TEST_DIR)):
    if not fn.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    path = os.path.join(TEST_DIR, fn)
    img_cv = cv2.imread(path)
    if img_cv is None:
        print(f"Warning: Could not read image {fn}")
        continue
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

    results = yolo_model.predict(img_rgb, conf=0.5, verbose=False)
    boxes = results[0].boxes

    if boxes is not None:
        for i in range(len(boxes)):
            # Get bounding box coordinates (axis-aligned)
            x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy().astype(int)
            gender = int(boxes.cls[i])  # 0 or 1
            color = (0, 255, 0) if gender == 0 else (255, 0, 0)

            # Crop + Resize for classification
            face = img_rgb[y1:y2, x1:x2]
            if face.size == 0:
                continue
            face_pil = Image.fromarray(face).resize(IMAGE_SIZE, Image.LANCZOS)
            face_np = np.array(face_pil)
            face_tensor = transform(image=face_np)['image'].unsqueeze(0).to(DEVICE)

            # Classification
            with torch.no_grad():
                output = classifier_model(face_tensor)
                pred_class = output.argmax(dim=1).item()
                label = -1 if pred_class == 15 else pred_class

            # Annotation
            text = f"{gender_map[gender]}, {class_names[label]}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.6
            thickness = 2

            # Calculate text size and position
            (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
            text_y = y1 - 10
            if text_y - text_height < 0:
                text_y = y1 + text_height + 5

            # Draw bounding box
            cv2.rectangle(img_cv, (x1, y1), (x2, y2), color, 2)
            # Draw filled rectangle for text background
            cv2.rectangle(
                img_cv,
                (x1, text_y - text_height - baseline),
                (x1 + text_width, text_y + baseline),
                color,
                thickness=cv2.FILLED
            )
            # Draw text
            cv2.putText(
                img_cv,
                text,
                (x1, text_y),
                font,
                font_scale,
                (255, 255, 255),
                thickness
            )

    # Save the image with annotations
    out_path = os.path.join(OUTPUT_FOLDER, fn)
    cv2.imwrite(out_path, img_cv)

print(f"✅ Annotated images saved in: {OUTPUT_FOLDER}")


100%|██████████| 425/425 [00:22<00:00, 18.48it/s]

✅ Annotated images saved in: /content/annotated_predictions_finall


In [40]:
import shutil

# Path to the folder you want to zip
folder_to_zip = "/content/annotated_predictions_finall"
zip_output_path = "/content/annotated_predictions_finall.zip"

# Create ZIP file
shutil.make_archive(base_name=zip_output_path.replace(".zip", ""), format='zip', root_dir=folder_to_zip)

print(f"✅ Zipped folder saved as: {zip_output_path}")


✅ Zipped folder saved as: /content/annotated_predictions_finall.zip


In [42]:
import shutil

# Path to the folder you want to zip
folder_to_zip = "/content/annotated_predictions"
zip_output_path = "/content/annotated_predictions.zip"

# Create ZIP file
shutil.make_archive(base_name=zip_output_path.replace(".zip", ""), format='zip', root_dir=folder_to_zip)

print(f"✅ Zipped folder saved as: {zip_output_path}")


✅ Zipped folder saved as: /content/annotated_predictions.zip
